In [3]:
import numpy as np
import cv2
import os
from PIL import Image, ImageEnhance, ImageOps
import torchvision.transforms.functional as F

In [4]:
def read_file(sn,tn):
	s = cv2.imread('source/'+sn+'.bmp')
	s = cv2.cvtColor(s,cv2.COLOR_BGR2LAB)
	t = cv2.imread('target/'+tn+'.bmp')
	t = cv2.cvtColor(t,cv2.COLOR_BGR2LAB)
	return s, t

In [5]:
def get_mean_and_std(x):
	x_mean, x_std = cv2.meanStdDev(x)
	x_mean = np.hstack(np.around(x_mean,2))
	x_std = np.hstack(np.around(x_std,2))
	return x_mean, x_std

In [6]:
# Make more target images from color augmentation

COLOR_TRANSFORMS = {
    'color_contrast_high': lambda img: ImageEnhance.Contrast(img).enhance(1.5),
    'color_contrast_low': lambda img: ImageEnhance.Contrast(img).enhance(0.5),
    'color_saturation_high': lambda img: ImageEnhance.Color(img).enhance(1.5),
    'color_saturation_low': lambda img: ImageEnhance.Color(img).enhance(0.5),
    'color_hue_p0_1': lambda img: F.adjust_hue(img, 0.1),
    'color_hue_n0_1': lambda img: F.adjust_hue(img, -0.1),
}



def augment_target_images(targets_dir):
    targets = os.listdir(targets_dir)
    for tn in targets:
        t = Image.open(os.path.join(targets_dir, tn))
        base_name, ext = os.path.splitext(tn)
        for transform_name, transform_func in COLOR_TRANSFORMS.items():
            t_aug = transform_func(t)
            aug_name = f"{base_name}_{transform_name}{ext}"
            t_aug.save(os.path.join(targets_dir, aug_name))
            # os.remove('target/'+tn+'_aug'+str(i)+'.bmp')


if __name__ == "__main__":
    augment_target_images('target')

In [7]:
def color_transfer(sources_dir, targets_dir):
	
	# read files from source and target folder and store names in 2 lists
	if not os.path.exists(sources_dir) or not os.path.exists(targets_dir):
		print("Source or target folder does not exist!")
	else:
		# make the sources list 6 times longer to match the number of target images (copy each source 6 times)
		sources = []
		for f in os.listdir(sources_dir):
			if f.endswith('.bmp'):
				for _ in range(6):
					sources.append(f[:-4])

		targets = [f[:-4] for f in os.listdir(targets_dir) if f.endswith('.bmp')]

	# delete files from result folder if exist
	if not os.path.exists('result'):
		os.makedirs('result')
	else:
		files = os.listdir('result')
		for file in files:
			os.remove('result/'+file)

    # Convert each source image with the corresponding target image
	for n in range(len(sources)):
		print("Converting picture"+str(n+1)+"...")
		s, t = read_file(sources[n],targets[n])
		s_mean, s_std = get_mean_and_std(s)
		t_mean, t_std = get_mean_and_std(t)

		height, width, channel = s.shape
		for i in range(0,height):
			for j in range(0,width):
				for k in range(0,channel):
					x = s[i,j,k]
					x = ((x-s_mean[k])*(t_std[k]/s_std[k]))+t_mean[k]
					# round or +0.5
					x = round(x)
					# boundary check
					x = 0 if x<0 else x
					x = 255 if x>255 else x
					s[i,j,k] = x

		s = cv2.cvtColor(s,cv2.COLOR_LAB2BGR)
		cv2.imwrite('result/rezultat_'+str(n+1)+'.bmp',s)

In [8]:
# main function
if __name__ == "__main__":
    color_transfer('source', 'target')

Converting picture1...
Converting picture2...
Converting picture3...
Converting picture4...
Converting picture5...
Converting picture6...
Converting picture7...
Converting picture8...
Converting picture9...
Converting picture10...
Converting picture11...
Converting picture12...
Converting picture13...
Converting picture14...
Converting picture15...
Converting picture16...
Converting picture17...
Converting picture18...
Converting picture19...
Converting picture20...
Converting picture21...
Converting picture22...
Converting picture23...
Converting picture24...
Converting picture25...
Converting picture26...
Converting picture27...
Converting picture28...
Converting picture29...
Converting picture30...
Converting picture31...
Converting picture32...
Converting picture33...
Converting picture34...
Converting picture35...
Converting picture36...
